In [1]:
import pandas as pd
import quandl
import datetime
import numpy as np

In [ ]:
#done with currency data
#data taken from https://www.ofx.com/en-au/forex-news/historical-exchange-rates/

In [7]:
def preprocess_series(df):
    df.index = pd.to_datetime(df.index)
    #slice the date range to 01-01-2017 -- 23-03-2017
    df = df['2018-02-23':'2018-05-16']
    #print(len(df))
    #print('dtype--',df.dtypes)
    #drop duplicate values
    #df = df.drop_duplicates(keep=False)
    #check datatype of the column,should only consist of numbers(fill with nans)
    df = df.apply(pd.to_numeric,errors='coerce')
    #print('null--',df.isnull().sum())
    #reindex if the date range not available.
    start = datetime.datetime(2018, 2, 23)
    end = datetime.datetime(2018, 5, 16)
    index = pd.date_range(start, end)
    df = df.reindex(index)
    #bfill or ffill to fill the values
    df = df.fillna(method='bfill')
    df = df.fillna(method='ffill')
    df = df.asfreq('60Min',method='pad')
    return pd.DataFrame(df)

In [3]:
#reading data
def get_exchange_data():
    #https://www.ofx.com/en-au/forex-news/historical-exchange-rates/
    eur = preprocess_series(pd.read_csv('US_EUR.csv',index_col='DATE'))
    cny = preprocess_series(pd.read_csv('US_CNY.csv',index_col='DATE'))
    chf = preprocess_series(pd.read_csv('US_CHF.csv',index_col='DATE'))
    gbp = preprocess_series(pd.read_csv('US_GBP.csv',index_col='DATE'))
    jpy = preprocess_series(pd.read_csv('US_JPN.csv',index_col='DATE'))
    exchange_data = pd.concat([eur,cny,chf,gbp,jpy],axis=1)
    exchange_data.columns = ['eur','cny','chf','gbp','jpy']
    return exchange_data


In [13]:
exchange_data = get_exchange_data()

In [14]:
exchange_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1969 entries, 2018-02-23 00:00:00 to 2018-05-16 00:00:00
Freq: 60T
Data columns (total 5 columns):
eur    1969 non-null float64
cny    1969 non-null float64
chf    1969 non-null float64
gbp    1969 non-null float64
jpy    1969 non-null float64
dtypes: float64(5)
memory usage: 172.3 KB


In [15]:
#quandl api
def get_crudeoil_data():
    oil_data = preprocess_series(pd.read_csv('crude_oil.csv',index_col='DATE'))
    return oil_data

In [16]:
crude_oil = get_crudeoil_data()

In [17]:
crude_oil.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1969 entries, 2018-02-23 00:00:00 to 2018-05-16 00:00:00
Freq: 60T
Data columns (total 1 columns):
crude_oil    1969 non-null float64
dtypes: float64(1)
memory usage: 30.8 KB


In [18]:
#quandl api
def get_gold_data():
    gold = preprocess_series(pd.read_csv('gold_price_test.csv',index_col='DATE'))
    return gold

In [19]:
gold_price = get_gold_data()

In [20]:
gold_price.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1969 entries, 2018-02-23 00:00:00 to 2018-05-16 00:00:00
Freq: 60T
Data columns (total 1 columns):
gold    1969 non-null float64
dtypes: float64(1)
memory usage: 30.8 KB


In [21]:
def get_stock_exchange_data():
    #https://query1.finance.yahoo.com/v7/finance/download/%5ESTOXX50E?period1=148318920
    SP500 = preprocess_series(pd.read_csv('SP500.csv',index_col='DATE')) 
    Nasdaq = preprocess_series(pd.read_csv('NASDAQ100.csv',index_col='DATE'))
    nikkei = preprocess_series(pd.read_csv('NIKKEI225.csv',index_col='DATE'))
    ftse = preprocess_series(pd.read_csv('FTSE100.csv',index_col='DATE'))
    eurostoxx = preprocess_series(pd.read_csv('Eurostox_50.csv',index_col='DATE'))
    Vix = preprocess_series(pd.read_csv('VIX.csv',index_col='DATE'))
    dow_jones = preprocess_series(pd.read_csv('DowJones.csv',index_col='DATE'))
    Sse = preprocess_series(pd.read_csv('SSE.csv',index_col='DATE'))
    
    stock_exchange_data = pd.concat([SP500,Nasdaq,nikkei,ftse,eurostoxx,Vix,dow_jones,Sse],axis=1)
    stock_exchange_data.columns = ['SP500','Nasdaq','Nikkei','Ftse','Eurostoxx','Vix','DowJones','Sse']
    return stock_exchange_data

In [22]:
stock_indices = get_stock_exchange_data()

In [23]:
stock_indices.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1969 entries, 2018-02-23 00:00:00 to 2018-05-16 00:00:00
Freq: 60T
Data columns (total 8 columns):
SP500        1969 non-null float64
Nasdaq       1969 non-null float64
Nikkei       1969 non-null float64
Ftse         1969 non-null float64
Eurostoxx    1969 non-null float64
Vix          1969 non-null float64
DowJones     1969 non-null float64
Sse          1969 non-null float64
dtypes: float64(8)
memory usage: 218.4 KB


In [24]:
def get_blockchain_info():
    quandl.ApiConfig.api_key = "6oZyLZYgnpY-E4zY4smC"
    start = datetime.datetime(2018, 2, 23)
    end = datetime.datetime(2018, 5, 16)
    freq='1H'
    index = pd.date_range(start, end,freq=freq)
    data_dict = dict()
    keys = [('BCHAIN/DIFF','difficulty'),('BCHAIN/AVBLS','average_block_size'),('BCHAIN/ATRCT','median_confirm_time'),\
            ('BCHAIN/MIREV','miner_revenue'),('BCHAIN/HRATE','hash_rate'),('BCHAIN/CPTRV','cost_%_trans_vol'),\
            ('BCHAIN/NTRBL','trans/block'),('BCHAIN/MKTCP','market_cap'),('BCHAIN/ETRVU','est_trans_vol_usd')] 
    for key,name in keys:
        preprocessed_series = preprocess_series(quandl.get(key, start_date='2018-02-01', end_date='2018-05-17'))
        flattened_array = np.concatenate(preprocessed_series.round(2).values).ravel()
        data_dict[name] = flattened_array
    blockchain_features = pd.DataFrame.from_dict(data_dict,orient='columns')
    blockchain_features.index = index
    confirmed_transactions = preprocess_series(pd.read_csv('Confirmed_Transactions.csv',index_col='DATE'))
    blockchain_features = pd.concat([blockchain_features,confirmed_transactions],axis=1)
    return blockchain_features

In [25]:
blockchain_data = get_blockchain_info()

In [26]:
blockchain_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1969 entries, 2018-02-23 00:00:00 to 2018-05-16 00:00:00
Freq: H
Data columns (total 10 columns):
average_block_size        1969 non-null float64
cost_%_trans_vol          1969 non-null float64
difficulty                1969 non-null float64
est_trans_vol_usd         1969 non-null float64
hash_rate                 1969 non-null float64
market_cap                1969 non-null float64
median_confirm_time       1969 non-null float64
miner_revenue             1969 non-null float64
trans/block               1969 non-null float64
Confirmed_transactions    1969 non-null int64
dtypes: float64(9), int64(1)
memory usage: 249.2 KB


In [27]:
test_dataset = pd.concat([exchange_data,crude_oil,gold_price,stock_indices,blockchain_data],axis=1)

In [28]:
test_dataset.head()

,eur,cny,chf,gbp,jpy,crude_oil,gold,SP500,Nasdaq,Nikkei,...,average_block_size,cost_%_trans_vol,difficulty,est_trans_vol_usd,hash_rate,market_cap,median_confirm_time,miner_revenue,trans/block,Confirmed_transactions
2018-02-23 00:00:00,0.81,6.34,0.94,0.72,106.65,63.52,1328.9,2747.3,6896.6,21892.78,...,0.92,0.75,3.007384e+12,2.511448e+09,22275181.57,1.676444e+11,9.12,18506552.05,1229.77,185193
2018-02-23 01:00:00,0.81,6.34,0.94,0.72,106.65,63.52,1328.9,2747.3,6896.6,21892.78,...,0.92,0.75,3.007384e+12,2.511448e+09,22275181.57,1.676444e+11,9.12,18506552.05,1229.77,185193
2018-02-23 02:00:00,0.81,6.34,0.94,0.72,106.65,63.52,1328.9,2747.3,6896.6,21892.78,...,0.92,0.75,3.007384e+12,2.511448e+09,22275181.57,1.676444e+11,9.12,18506552.05,1229.77,185193
2018-02-23 03:00:00,0.81,6.34,0.94,0.72,106.65,63.52,1328.9,2747.3,6896.6,21892.78,...,0.92,0.75,3.007384e+12,2.511448e+09,22275181.57,1.676444e+11,9.12,18506552.05,1229.77,185193
2018-02-23 04:00:00,0.81,6.34,0.94,0.72,106.65,63.52,1328.9,2747.3,6896.6,21892.78,...,0.92,0.75,3.007384e+12,2.511448e+09,22275181.57,1.676444e+11,9.12,18506552.05,1229.77,185193


In [29]:
test_dataset.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1969 entries, 2018-02-23 00:00:00 to 2018-05-16 00:00:00
Freq: 60T
Data columns (total 25 columns):
eur                       1969 non-null float64
cny                       1969 non-null float64
chf                       1969 non-null float64
gbp                       1969 non-null float64
jpy                       1969 non-null float64
crude_oil                 1969 non-null float64
gold                      1969 non-null float64
SP500                     1969 non-null float64
Nasdaq                    1969 non-null float64
Nikkei                    1969 non-null float64
Ftse                      1969 non-null float64
Eurostoxx                 1969 non-null float64
Vix                       1969 non-null float64
DowJones                  1969 non-null float64
Sse                       1969 non-null float64
average_block_size        1969 non-null float64
cost_%_trans_vol          1969 non-null float64
difficulty                1969 

In [30]:
test_dataset.to_csv('test_data.csv')